In [ ]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

In [ ]:
# establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname = "dw_cs", 
    user = "postgres", 
    #host= 'localhost',
    host= '172.31.160.1',
    password = "test31415",
    port = 5432
)

In [ ]:
#with conn.cursor() as cur:
    
#    cur.execute("DELETE FROM orders;")
#    cur.execute("DELETE FROM order_items;")    


In [ ]:
def create_partition(table, attribute, init_month, end_month, year):
    if end_month==12:
        end_month_part=1
        new_year=year+1
    else:
        end_month_part= str(int(end_month)+1)
        new_year=year
    
    query=f"""CREATE TABLE  {table}_m{init_month}_m{end_month}_{year} (
    CHECK ({attribute} >= '{year}-{init_month}-01' AND {attribute} < '{new_year}-{end_month_part}-01'))
    INHERITS ({table});"""
    return query

In [ ]:
for year in range(1992,1999):
    for month in range(1,12,3):
        query=create_partition('orders', 'o_orderdate', month, month+2, year)
        with conn.cursor() as cursor:
            cursor.execute(query)
            conn.commit()
            print('Partition created')